In [ ]:
import pymongo # import the library
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency

In [ ]:
client = MongoClient("mongodb://localhost:27017") # connect to the database
db = client['3675Local'] # connect to the database

I want to create a list of valid tconst. I will do this by creating a collection of movies with a runtime between 60 to 220 mins. Then I will drop all records with empty runtime. 

Lucky for me I already created a list of valid tconsts that are almost there. I just need to drop all that have empty runtimes

In [ ]:
path = r'3675IMDbData\title.basics.tsv.gz'
df = pd.read_csv(path, sep='\t')

In [ ]:
# load the valid tconsts
path = r'valid_tconsts.csv'
valid_tconsts_df = pd.read_csv(path)

In [ ]:
# join the two df to create a new df with only the valid tconsts
df = pd.merge(df, valid_tconsts_df, on='tconst', how='inner')

In [17]:
# drop all records with "\N" value in runtimeMinutes
df = df[df['runtimeMinutes'] != '\\N']

# drop all records with "\N" value in genres
df = df[df['genres'] != '\\N']

Convert the runtimeMinutes and startYear to numerical

In [ ]:
df.loc[:, "runtimeMinutes"] = pd.to_numeric(df["runtimeMinutes"], errors="coerce")
df.loc[:, "startYear"] = pd.to_numeric(df["startYear"], errors="coerce")

In [20]:
# Turn into a dictionary for mongodb
data = df.to_dict(orient='records')

Add to DB

In [25]:
collection = db["title.basics_cleaned"] # create a collection

# insert the data into the collection
collection.insert_many(data)
print(len(data))
# 269360

269360


**Create a csv of valid_non_empty_tconst**

In [27]:
# Query the collection - this returns a cursor
cursor = db["title.basics_cleaned"].find()

# Convert the cursor to a list of documents, then create a DataFrame
list = list(cursor)
list_df = pd.DataFrame(list)

# Drop the MongoDB autogenerated '_id' column
if '_id' in list_df.columns:
    list_df = list_df.drop('_id', axis=1)


# Get the valid tconst values from Movies_only_basics
valid_nonzero_tconsts = list_df["tconst"].unique()

# Convert to text file for use later 
valid_nonzero_tconsts = pd.DataFrame(valid_nonzero_tconsts, columns=["tconst"])
valid_nonzero_tconsts.to_csv("valid_nonzero_tconsts.csv", index=False)

**Join title.ratings and valid_nonzero_tconsts**

In [32]:
# load title.ratings.tsv.gz
path = r'3675IMDbData\title.ratings.tsv.gz'
ratings_df = pd.read_csv(path, sep='\t')

# join the two df to create a new df 

df = pd.merge(valid_nonzero_tconsts, ratings_df, on='tconst', how='inner')


In [ ]:
# check for missing values
results = df[df['averageRating'] == '\\N']
print(results)

**Join title.crew and valid_nonzero_tconsts**

In [36]:
path = r'3675IMDbData\title.crew.tsv.gz'
crew_df = pd.read_csv(path, sep='\t')

# join the two df to create a new df
df = pd.merge(valid_nonzero_tconsts, crew_df, on='tconst', how='inner')

In [ ]:
# Create a new collection
collection = db["title.crew_cleaned"]
data = df.to_dict(orient='records')

# insert the data into the collection
collection.insert_many(data)
print(len(data))
# 269357

269357


**Join title.crew and valid_nonzero_tconsts**